In [1]:
import os
from langchain_groq import ChatGroq
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-70b-8192")


In [2]:
# Build Currancy Converter
import requests

os.environ['Exchange_Rate_API'] = os.getenv("Exchange_Rate_API")

In [3]:
from langchain_core.tools import tool
@tool
def get_conversion_factor(base_currency:str,target_currency:str)->float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    response = requests.get(url=url)
    return response.json()



# @tool
# def get_conversion_factor(base_currency: str, target_currency: str) -> float:
#   """
#   This function fetches the currency conversion factor between a given base currency and a target currency
#   """
#   url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

#   response = requests.get(url)

#   return response.json()


In [4]:
get_conversion_factor.invoke({"base_currency":"USD","target_currency":'inr'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1751587201,
 'time_last_update_utc': 'Fri, 04 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1751673601,
 'time_next_update_utc': 'Sat, 05 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4393}

In [5]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [6]:
convert({"base_currency_value":10,"conversion_rate":25})

C:\Users\ANKIT\AppData\Local\Temp\ipykernel_13576\1505291774.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  convert({"base_currency_value":10,"conversion_rate":25})


250.0

In [7]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [8]:
llm_with_tools = model.bind_tools([get_conversion_factor, convert])


In [9]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage('convert 10 Inr into USD')]

In [10]:
ai_message = llm_with_tools.invoke(messages)
ai_message.tool_calls


[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'qfktevm32',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '92h4qz74v',
  'type': 'tool_call'}]